In [ ]:
import pandas as pd

df = pd.read_csv('data/data2.tsv', delimiter = '\t')

df

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

imgs = np.array([ img_to_array(load_img(i)) for i in df['image'].values ])

imgs.shape

In [ ]:
cols = ['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']

dfl = df[cols].astype(np.float64)

for i in range(0, len(cols), 2):
    dfl[cols[i]] /= imgs.shape[2]
    dfl[cols[i + 1]] /= imgs.shape[1]

labels = dfl.values
labels

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, Reshape
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.normalization import BatchNormalization

input = Input(shape = imgs.shape[1:])

x = BatchNormalization()(input)

x = Conv2D(16, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Conv2D(128, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(128, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

def flatten_conv(d):
    cv = Conv2D(16, 1, activation = 'relu')
    d = cv(d)

    sh1 = cv.output_shape

    ft = Flatten()
    d = ft(d)

    sh2 = ft.output_shape
    
    d = Dense(int(sh2[-1] / 2), activation = 'relu')(d)
    d = Dense(int(sh2[-1] / 4), activation = 'relu')(d)
    d = Reshape(tuple(sh1[1:-1]) + (4,))(d)

    d = BatchNormalization()(d)
    d = Dropout(0.2)(d)

    d = Conv2D(32, 3, padding = 'same', activation = 'relu')(d)
    d = Conv2D(32, 3, padding = 'same', activation = 'relu')(d)

    d = BatchNormalization()(d)
    d = Dropout(0.2)(d)
    
    return d


for i in range(2):
    x = flatten_conv(x)

x = Conv2D(16, 1, activation = 'relu')(x)
x = Flatten()(x)

x = Dense(256, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)

output = Dense(8, activation = 'sigmoid')(x)

model = Model(inputs = input, outputs = output)

#model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mae'])
model.compile(loss = 'mae', optimizer = 'adam', metrics = ['mse'])

model.summary()

In [ ]:
hist = model.fit(imgs, labels, epochs = 50, batch_size = 10)
hist

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.plot(hist.history['loss'])

In [ ]:
model.save('model/cnn-3_0.h5')

In [ ]:
import cv2

def scale(d):
    for i in range(0, len(d), 2):
        d[i] *= imgs.shape[2]
        d[i + 1] *= imgs.shape[1]

    return d
        

def predict(i, s = 8.0):
    plt.rcParams['figure.figsize'] = (s, s)
    
    r = scale(np.copy(labels[i])).astype(int)
    p = scale(model.predict(np.array([imgs[i]]))[0]).astype(int)

    print(r)
    print(p)
    
    img = imgs[i].astype(int)

    line_size = 3
    r_color = (0, 255, 0)
    p_color = (255, 0, 0)
    
    for i in range(0, len(r), 4):
        img = cv2.line(img, (r[i], r[i + 1]), (r[i + 2], r[i + 3]), r_color, line_size)
        img = cv2.line(img, (p[i], p[i + 1]), (p[i + 2], p[i + 3]), p_color, line_size)
    
    plt.imshow(img)


In [ ]:
predict(0)

In [ ]:
predict(1)

In [ ]:
predict(2)

In [ ]:
predict(3)